In [ ]:
import pandas as pd
import datetime
from itertools import product

## Prepare usable data

In [ ]:
# Manually analyzed and queried the usable dates from postgrey
# The following code only works in local machine as it needs to connect to postgrey
'''
# Connect to the postgrey server using your IP address, username and password
Hostname= "localhost"
HostIP: "192.168.1.155" #Santhosh's IP
Dbname= "DockWatchDB"
Username= "postgres" #Santhosh
Password= "Niranjan_99" #Santhosh_99
Port= 5432
Schema= "mastertables"

conn = psycopg2.connect(
    host= Hostname,
    dbname= Dbname,
    user= Username,
    password= Password,
    port= Port
)

query= "SELECT 	* FROM  Preprocessed.\"Bcycle_Status_TD_Preprocessed\" WHERE DATE BETWEEN '2023-10-31' AND '2023-11-06';"
df = pd.read_sql_query(query, conn)

# All selected Tuesday to Saturday data  are accurate and complete
# Selected Sunday (2023-11-05) data is available only till 17:26:10, remaining are just the duplicates of the last availabe status
# Selected Monday (2023-11-06) data is accureate only after 17:40:44, hence the ones before it just the replicate of Tuesday's (2023-10-31)

# Always close the connection at the end
conn.close()

print(df.head(10))

#df.to_csv("C:\Users\admin\OneDrive - UCB-O365\01. Study\03. CUB Courses\01. Undergoing Courses\03. CSCI 5502- Data Mining\04. Project\01. Data\02. Preprocessed Data\Sample_Usable_Preprocessed_Data.csv",index=False)
df.to_csv('Sample_Usable_Preprocessed_Data.csv',index=False)
'''

In [ ]:
# The usable data is que
df= pd.read_csv('/content/Sample_Usable_Preprocessed_Data.csv')
df.head()

,station_last_updated,status_last_reported,station_id,station_name,station_address,station_longitude,station_latitude,station_is_returning,station_is_renting,station_is_installed,...,datatime_mtd,date,year,month,day,day_of_week,time,hour,minute,second
0,1698732017,1698732020,bcycle_boulder_3589,30th & Marine,1570 30th St.,-105.25354,40.01356,1,1,1,...,2023-10-31 00:00:20,2023-10-31,2023,10,31,Tuesday,00:00:20,0,0,20
1,1698732017,1698732020,bcycle_boulder_3379,28th & Mapleton,28th & Mapleton,-105.25856,40.02620,1,1,1,...,2023-10-31 00:00:20,2023-10-31,2023,10,31,Tuesday,00:00:20,0,0,20
2,1698732017,1698732020,bcycle_boulder_3318,18th & Euclid,1725 Euclid,-105.26981,40.00584,1,1,1,...,2023-10-31 00:00:20,2023-10-31,2023,10,31,Tuesday,00:00:20,0,0,20
3,1698732017,1698732020,bcycle_boulder_2875,35th & Colorado,35th & Colorado Streets,-105.24793,40.00767,1,1,1,...,2023-10-31 00:00:20,2023-10-31,2023,10,31,Tuesday,00:00:20,0,0,20
4,1698732017,1698732020,bcycle_boulder_2766,13th & Arapahoe,Boulder Creek Path,-105.27764,40.01450,1,1,1,...,2023-10-31 00:00:20,2023-10-31,2023,10,31,Tuesday,00:00:20,0,0,20


In [ ]:
# Round off the datatime to the previous ideal time interval (3 or 1 min)
time_interval_to_round= 3
df['datatime_mtd'] = pd.to_datetime(df['datatime_mtd'], format='%Y-%m-%d %H:%M:%S')
df['datatime_rnd'] = df['datatime_mtd'].apply(lambda x: x - pd.to_timedelta(x.minute % time_interval_to_round, unit='m') - pd.to_timedelta(x.second, unit='s')) # This rounds off the minute and makes the seconds zero

# However, rounding off values will create duplicates, so the below line will take only the first entry of every duplicate. For example, Minute 3, 4, and 5 will become 3- so this takes the first entry (3) as that is the closest representation
df.drop_duplicates(subset=['station_id', 'datatime_rnd'], keep='first', inplace=True)
  # print("Number of duplicates:", df.duplicated(subset=['station_id', 'datatime_rnd']).sum()) # To check duplicates
  # print("Number of duplicates per combination:\n\n", df.groupby(['station_id', 'datatime_rnd']).size().reset_index(name='count_of_duplicates').sort_values(by='count_of_duplicates', ascending=False)) # To check duplicates per combination
'''
  #Alternate way: Just make the seconds zero and multiply the numerator of minute divided by [req_time_interval] with [req_time_interval]

  req_time_interval= 3
  df[min_rounded] = int(df['min']/req_time_interval) * req_time_interval
'''
# Split dates and times of from the rounded off datatime

df['date_rnd']= df['datatime_rnd'].dt.date
df['year_rnd'] = df['datatime_rnd'].dt.year
df['month_rnd'] = df['datatime_rnd'].dt.month
df['day_rnd'] = df['datatime_rnd'].dt.day
df['day_of_week_rnd'] = df['datatime_rnd'].dt.day_name()
df['time_rnd']= df['datatime_rnd'].dt.time
df['hour_rnd'] = df['datatime_rnd'].dt.hour
df['minute_rnd'] = df['datatime_rnd'].dt.minute
df['second_rnd'] = df['datatime_rnd'].dt.second
df.drop(columns = ['date',"year","month","day","day_of_week","time","hour","minute","second"],inplace=True)

df.head(5)

,station_last_updated,status_last_reported,station_id,station_name,station_address,station_longitude,station_latitude,station_is_returning,station_is_renting,station_is_installed,...,datatime_rnd,date_rnd,year_rnd,month_rnd,day_rnd,day_of_week_rnd,time_rnd,hour_rnd,minute_rnd,second_rnd
0,1698732017,1698732020,bcycle_boulder_3589,30th & Marine,1570 30th St.,-105.25354,40.01356,1,1,1,...,2023-10-31,2023-10-31,2023,10,31,Tuesday,00:00:00,0,0,0
1,1698732017,1698732020,bcycle_boulder_3379,28th & Mapleton,28th & Mapleton,-105.25856,40.02620,1,1,1,...,2023-10-31,2023-10-31,2023,10,31,Tuesday,00:00:00,0,0,0
2,1698732017,1698732020,bcycle_boulder_3318,18th & Euclid,1725 Euclid,-105.26981,40.00584,1,1,1,...,2023-10-31,2023-10-31,2023,10,31,Tuesday,00:00:00,0,0,0
3,1698732017,1698732020,bcycle_boulder_2875,35th & Colorado,35th & Colorado Streets,-105.24793,40.00767,1,1,1,...,2023-10-31,2023-10-31,2023,10,31,Tuesday,00:00:00,0,0,0
4,1698732017,1698732020,bcycle_boulder_2766,13th & Arapahoe,Boulder Creek Path,-105.27764,40.01450,1,1,1,...,2023-10-31,2023-10-31,2023,10,31,Tuesday,00:00:00,0,0,0


In [ ]:
df_usage= df.copy()

## Prepare the ideal dataframe

In [ ]:
# Create ideal dataframe

# Decide the start and end datetime
start_datetime = '2023-10-31 00:00:00' # str(df['date_rnd'].min()) + " 00:00:00"
end_datetime = '2023-11-06 23:57:00' # str(df['date_rnd'].max()) + " 23:57:00"

# Create a datetime range with a 3-minute gap interval
datetime_range = pd.date_range(start=start_datetime, end=end_datetime, freq='3T')

# List of 54 unique station IDs
station_ids = ['bcycle_boulder_1855', 'bcycle_boulder_1858', 'bcycle_boulder_1859', 'bcycle_boulder_1860', 'bcycle_boulder_1861', 'bcycle_boulder_1866', 'bcycle_boulder_1867', 'bcycle_boulder_1869', 'bcycle_boulder_1870', 'bcycle_boulder_1871', 'bcycle_boulder_1872', 'bcycle_boulder_1873', 'bcycle_boulder_1943', 'bcycle_boulder_2021', 'bcycle_boulder_2022', 'bcycle_boulder_2132', 'bcycle_boulder_2144', 'bcycle_boulder_2161', 'bcycle_boulder_2182', 'bcycle_boulder_2198', 'bcycle_boulder_2756', 'bcycle_boulder_2757', 'bcycle_boulder_2759', 'bcycle_boulder_2760', 'bcycle_boulder_2761', 'bcycle_boulder_2762', 'bcycle_boulder_2763', 'bcycle_boulder_2764', 'bcycle_boulder_2765', 'bcycle_boulder_2766', 'bcycle_boulder_2767', 'bcycle_boulder_2768', 'bcycle_boulder_2769', 'bcycle_boulder_2770', 'bcycle_boulder_2771', 'bcycle_boulder_2875', 'bcycle_boulder_3318', 'bcycle_boulder_3379', 'bcycle_boulder_3589', 'bcycle_boulder_3675', 'bcycle_boulder_3676', 'bcycle_boulder_3876', 'bcycle_boulder_3894', 'bcycle_boulder_4091', 'bcycle_boulder_4142', 'bcycle_boulder_4656', 'bcycle_boulder_4657', 'bcycle_boulder_7305', 'bcycle_boulder_7314', 'bcycle_boulder_7327', 'bcycle_boulder_7345', 'bcycle_boulder_7346', 'bcycle_boulder_7387', 'bcycle_boulder_7393']
# station_ids = df['station_id'].unique()

# Creating combinations
combinations = list(product(datetime_range, station_ids))

# Creating the ideal dataframe using the combinations
df_ideal = pd.DataFrame(combinations, columns=['Datatime_ideal', 'station_id_ideal'])

print("Is the size of ideal datasets [number of datetime] * [54 stations]:", df_ideal.shape[0] == len(df_ideal['Datatime_ideal'].unique()) * len(df_ideal['station_id_ideal'].unique()) )
df_ideal.head()

Is the size of ideal datasets [number of datetime] * [54 stations]: True


,Datatime_ideal,station_id_ideal
0,2023-10-31,bcycle_boulder_1855
1,2023-10-31,bcycle_boulder_1858
2,2023-10-31,bcycle_boulder_1859
3,2023-10-31,bcycle_boulder_1860
4,2023-10-31,bcycle_boulder_1861


# Merge them both and find missing values (datatime x stationid)

In [ ]:
print("The number of rows that should be null:",df_ideal.shape[0] - df_usage.shape[0])

The number of rows that should be null: 26136


In [ ]:
df_merged= pd.merge(df_ideal, df_usage, how='left', left_on=['Datatime_ideal','station_id_ideal'], right_on=['datatime_rnd','station_id'])
df_merged.shape

(181440, 32)

In [ ]:
df_merged.isna().sum().max()

26136

In [ ]:
# Seperate null and not-null datasets
df_non_null= df_merged[df_merged['datatime_rnd'].notna()]
df_null= df_merged[df_merged['datatime_rnd'].isna()]

print(df_non_null.shape)
print(df_null.shape)

(155304, 32)
(26136, 32)


In [ ]:
df_null['date_null'] = df_null['Datatime_ideal'].dt.date
df_null['time_null'] = df_null['Datatime_ideal'].dt.time
df_null['weekday_null'] = df_null['Datatime_ideal'].dt.day_name()

In [ ]:
df_null[['date_null','weekday_null']].value_counts()

date_null   weekday_null
2023-11-06  Monday          19062
2023-11-05  Sunday           7074
dtype: int64

# Replace the missing values

In [ ]:
# Replacing null entries of Monday (2023-11-06) with Tuesday's (2023-10-31) data

df_null_monday= df_null [df_null['weekday_null']=='Monday'][['Datatime_ideal','station_id_ideal','time_null']]
monday_replace_data= df_usage[df_usage['date_rnd']==datetime.date(year=2023, month= 10, day= 31)]

df_replaced_monday= pd.merge(df_null_monday, monday_replace_data, how='left', left_on=['time_null','station_id_ideal'], right_on=['time_rnd','station_id'])
df_replaced_monday.isna().any().sum()

0

In [ ]:
# Fixing the date columns of df_replaced_monday

df_replaced_monday.drop(columns=['datatime_rnd','date_rnd',"year_rnd","month_rnd","day_rnd","day_of_week_rnd","time_rnd","hour_rnd","minute_rnd","second_rnd"], inplace= True)
df_replaced_monday.rename(columns={"Datatime_ideal": "datatime_rnd"}, inplace=True)

df_replaced_monday['date_rnd']= df_replaced_monday['datatime_rnd'].dt.date
df_replaced_monday['year_rnd'] = df_replaced_monday['datatime_rnd'].dt.year
df_replaced_monday['month_rnd'] = df_replaced_monday['datatime_rnd'].dt.month
df_replaced_monday['day_rnd'] = df_replaced_monday['datatime_rnd'].dt.day
df_replaced_monday['day_of_week_rnd'] = df_replaced_monday['datatime_rnd'].dt.day_name()
df_replaced_monday['time_rnd']= df_replaced_monday['datatime_rnd'].dt.time
df_replaced_monday['hour_rnd'] = df_replaced_monday['datatime_rnd'].dt.hour
df_replaced_monday['minute_rnd'] = df_replaced_monday['datatime_rnd'].dt.minute
df_replaced_monday['second_rnd'] = df_replaced_monday['datatime_rnd'].dt.second

In [ ]:
# Replacing null entries of Sunday (2023-11-05) with just the duplicates of it's last availabe time's status

df_null_sunday= df_null [df_null['weekday_null']=='Sunday'][['Datatime_ideal','station_id_ideal','time_null']]
sunday_replace_data= df_usage[df_usage['datatime_rnd'] == '2023-11-05 17:24:00'] # The last available data of Sunday (2023-11-05)

df_replaced_sunday= pd.merge(df_null_sunday, sunday_replace_data, how='left', left_on=['station_id_ideal'], right_on=['station_id'])
df_replaced_sunday.isna().any().sum()

0

In [ ]:
# Fixing the date columns of df_replaced_sunday

df_replaced_sunday.drop(columns=['datatime_rnd','date_rnd',"year_rnd","month_rnd","day_rnd","day_of_week_rnd","time_rnd","hour_rnd","minute_rnd","second_rnd"], inplace= True)
df_replaced_sunday.rename(columns={"Datatime_ideal": "datatime_rnd"}, inplace=True)

df_replaced_sunday['date_rnd']= df_replaced_sunday['datatime_rnd'].dt.date
df_replaced_sunday['year_rnd'] = df_replaced_sunday['datatime_rnd'].dt.year
df_replaced_sunday['month_rnd'] = df_replaced_sunday['datatime_rnd'].dt.month
df_replaced_sunday['day_rnd'] = df_replaced_sunday['datatime_rnd'].dt.day
df_replaced_sunday['day_of_week_rnd'] = df_replaced_sunday['datatime_rnd'].dt.day_name()
df_replaced_sunday['time_rnd']= df_replaced_sunday['datatime_rnd'].dt.time
df_replaced_sunday['hour_rnd'] = df_replaced_sunday['datatime_rnd'].dt.hour
df_replaced_sunday['minute_rnd'] = df_replaced_sunday['datatime_rnd'].dt.minute
df_replaced_sunday['second_rnd'] = df_replaced_sunday['datatime_rnd'].dt.second

In [ ]:
df_replaced_monday.shape[0] + df_replaced_sunday.shape[0]

26136

# Combine the replaced data with the non-null data and select only the required columns

In [ ]:
required_cols = ['station_id','datatime_mtd', 'datatime_rnd','date_rnd', 'year_rnd', 'month_rnd', 'day_rnd', 'day_of_week_rnd', 'time_rnd', 'hour_rnd', 'minute_rnd', 'second_rnd','station_name', 'station_address', 'station_longitude', 'station_latitude', 'station_is_returning','station_is_renting', 'station_is_installed', 'station_type','docks_available', 'bikes_available', 'electic_bikes_available', 'smart_bikes_available', 'classic_bikes_available', 'station_capacity']

In [ ]:
# Select only the req cols in all datasets
df1= df_non_null[required_cols]
df2= df_replaced_monday[required_cols]
df3= df_replaced_sunday[required_cols]

df_sample_data_final = pd.concat([df1,df2,df3])
df_sample_data_final.sort_values(by= ['datatime_rnd','station_id'], ascending=True,inplace=True)
df_sample_data_final

,station_id,datatime_mtd,datatime_rnd,date_rnd,year_rnd,month_rnd,day_rnd,day_of_week_rnd,time_rnd,hour_rnd,...,station_is_returning,station_is_renting,station_is_installed,station_type,docks_available,bikes_available,electic_bikes_available,smart_bikes_available,classic_bikes_available,station_capacity
0,bcycle_boulder_1855,2023-10-31 00:00:20,2023-10-31 00:00:00,2023-10-31,2023.0,10.0,31.0,Tuesday,00:00:00,0.0,...,1.0,1.0,1.0,Kiosk and Station,15.0,0.0,0.0,0.0,0.0,15.0
1,bcycle_boulder_1858,2023-10-31 00:00:20,2023-10-31 00:00:00,2023-10-31,2023.0,10.0,31.0,Tuesday,00:00:00,0.0,...,1.0,1.0,1.0,Kiosk and Station,15.0,0.0,0.0,0.0,0.0,15.0
2,bcycle_boulder_1859,2023-10-31 00:00:20,2023-10-31 00:00:00,2023-10-31,2023.0,10.0,31.0,Tuesday,00:00:00,0.0,...,1.0,1.0,1.0,Kiosk and Station,14.0,0.0,0.0,0.0,0.0,14.0
3,bcycle_boulder_1860,2023-10-31 00:00:20,2023-10-31 00:00:00,2023-10-31,2023.0,10.0,31.0,Tuesday,00:00:00,0.0,...,1.0,1.0,1.0,Kiosk and Station,13.0,0.0,0.0,0.0,0.0,13.0
4,bcycle_boulder_1861,2023-10-31 00:00:20,2023-10-31 00:00:00,2023-10-31,2023.0,10.0,31.0,Tuesday,00:00:00,0.0,...,1.0,1.0,1.0,Kiosk and Station,9.0,2.0,2.0,0.0,0.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181435,bcycle_boulder_7327,2023-11-06 23:57:04,2023-11-06 23:57:00,2023-11-06,2023.0,11.0,6.0,Monday,23:57:00,23.0,...,1.0,1.0,1.0,3.0 Dock Station,4.0,4.0,4.0,0.0,0.0,8.0
181436,bcycle_boulder_7345,2023-11-06 23:57:04,2023-11-06 23:57:00,2023-11-06,2023.0,11.0,6.0,Monday,23:57:00,23.0,...,1.0,1.0,1.0,3.0 Dock Station,5.0,1.0,1.0,0.0,0.0,6.0
181437,bcycle_boulder_7346,2023-11-06 23:57:04,2023-11-06 23:57:00,2023-11-06,2023.0,11.0,6.0,Monday,23:57:00,23.0,...,1.0,1.0,1.0,3.0 Dock Station,3.0,3.0,3.0,0.0,0.0,6.0
181438,bcycle_boulder_7387,2023-11-06 23:57:04,2023-11-06 23:57:00,2023-11-06,2023.0,11.0,6.0,Monday,23:57:00,23.0,...,1.0,1.0,1.0,3.0 Dock Station,0.0,5.0,5.0,0.0,0.0,5.0


# Calculate the new_docks and new_bikes available

Null replaced data might not have continutiy in the data availablity and result in high # of new_docks and new_bikes available

In [ ]:
# Create a grouped dataframe and calculate new docks and bikes available for every station at every datetime

# Sort the date by datetime
df_sample_data_final = df_sample_data_final.sort_values(by='datatime_rnd')

# Group by station_id and calculate # new cycles and new docks available
  # Why group by? If not the new-docks_avlb will be calculated by just subtracting the above row- which will be a different station_id
grouped_df = df_sample_data_final.groupby(['station_id', 'datatime_rnd']).agg({'docks_available': 'sum', 'bikes_available': 'sum'}).reset_index()

# Calculate the differences with the previous row and handle the cases when it's NULL or <= 0
grouped_df['new_docks_available'] = grouped_df['docks_available'].diff().fillna(0).clip(lower=0).astype(int)
grouped_df['new_bikes_available'] = grouped_df['bikes_available'].diff().fillna(0).clip(lower=0).astype(int)

grouped_df.head(10)

,station_id,datatime_rnd,docks_available,bikes_available,new_docks_available,new_bikes_available
0,bcycle_boulder_1855,2023-10-31 00:00:00,15.0,0.0,0,0
1,bcycle_boulder_1855,2023-10-31 00:03:00,15.0,0.0,0,0
2,bcycle_boulder_1855,2023-10-31 00:06:00,15.0,0.0,0,0
3,bcycle_boulder_1855,2023-10-31 00:09:00,15.0,0.0,0,0
4,bcycle_boulder_1855,2023-10-31 00:12:00,15.0,0.0,0,0
5,bcycle_boulder_1855,2023-10-31 00:15:00,15.0,0.0,0,0
6,bcycle_boulder_1855,2023-10-31 00:18:00,15.0,0.0,0,0
7,bcycle_boulder_1855,2023-10-31 00:21:00,15.0,0.0,0,0
8,bcycle_boulder_1855,2023-10-31 00:24:00,15.0,0.0,0,0
9,bcycle_boulder_1855,2023-10-31 00:27:00,15.0,0.0,0,0


In [ ]:
# Combine the grouped data containing new bikes/docks available with the actual data

# Outer join tables
df_final= pd.merge(df_sample_data_final, grouped_df, left_on=['station_id','datatime_rnd'], right_on=['station_id','datatime_rnd'], how='outer')

if(df_final.isna().sum().sum() == 0):
  print("Join Successfull!!")
else:
  print("Join Failed: Nulls in the data indicates the grouped data has less number of entries")

# Rename, remove unwanted and rearrange columns
df_final.rename(columns={'docks_available_x': 'docks_available', 'bikes_available_x': 'bikes_available'}, inplace=True)
df_final.drop(columns=['docks_available_y','docks_available_y'],inplace=True)

Join Successfull!!


In [ ]:
df_final

,station_id,datatime_mtd,datatime_rnd,date_rnd,year_rnd,month_rnd,day_rnd,day_of_week_rnd,time_rnd,hour_rnd,...,station_type,docks_available,bikes_available,electic_bikes_available,smart_bikes_available,classic_bikes_available,station_capacity,bikes_available_y,new_docks_available,new_bikes_available
0,bcycle_boulder_1855,2023-10-31 00:00:20,2023-10-31 00:00:00,2023-10-31,2023.0,10.0,31.0,Tuesday,00:00:00,0.0,...,Kiosk and Station,15.0,0.0,0.0,0.0,0.0,15.0,0.0,0,0
1,bcycle_boulder_2766,2023-10-31 00:00:20,2023-10-31 00:00:00,2023-10-31,2023.0,10.0,31.0,Tuesday,00:00:00,0.0,...,Kiosk and Station,13.0,0.0,0.0,0.0,0.0,13.0,0.0,5,0
2,bcycle_boulder_2767,2023-10-31 00:00:20,2023-10-31 00:00:00,2023-10-31,2023.0,10.0,31.0,Tuesday,00:00:00,0.0,...,Kiosk and Station,14.0,0.0,0.0,0.0,0.0,14.0,0.0,4,0
3,bcycle_boulder_2768,2023-10-31 00:00:20,2023-10-31 00:00:00,2023-10-31,2023.0,10.0,31.0,Tuesday,00:00:00,0.0,...,Kiosk and Station,9.0,4.0,4.0,0.0,0.0,13.0,4.0,0,4
4,bcycle_boulder_2769,2023-10-31 00:00:20,2023-10-31 00:00:00,2023-10-31,2023.0,10.0,31.0,Tuesday,00:00:00,0.0,...,Kiosk and Station,2.0,8.0,8.0,0.0,0.0,10.0,8.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181435,bcycle_boulder_2757,2023-11-06 23:57:04,2023-11-06 23:57:00,2023-11-06,2023.0,11.0,6.0,Monday,23:57:00,23.0,...,Kiosk and Station,16.0,2.0,2.0,0.0,0.0,18.0,2.0,0,0
181436,bcycle_boulder_2759,2023-11-06 23:57:04,2023-11-06 23:57:00,2023-11-06,2023.0,11.0,6.0,Monday,23:57:00,23.0,...,Kiosk and Station,10.0,5.0,5.0,0.0,0.0,15.0,5.0,0,0
181437,bcycle_boulder_2760,2023-11-06 23:57:04,2023-11-06 23:57:00,2023-11-06,2023.0,11.0,6.0,Monday,23:57:00,23.0,...,Kiosk and Station,11.0,1.0,1.0,0.0,0.0,12.0,1.0,0,0
181438,bcycle_boulder_1873,2023-11-06 23:57:04,2023-11-06 23:57:00,2023-11-06,2023.0,11.0,6.0,Monday,23:57:00,23.0,...,Kiosk and Station,1.0,8.0,8.0,0.0,0.0,9.0,8.0,0,0


# Calculate bike and dock wait times

In [ ]:
df= df_final.copy()

# Sort the data by datetime
df.sort_values(by=['datatime_rnd'],inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

In [ ]:
# Calculate the next available bike and dock time for each station

def cal_next_avl_time(all_times, avl_times):
  next_avl_time=[]
  i=0
  j=0
  while j<len(avl_times) and i<len(all_times):
    if (all_times[i]< avl_times[j]):
      next_avl_time.append(avl_times[j])
      i+=1
    else:
      j+=1
  next_avl_time += [None]*(len(all_times)-len(next_avl_time))
  return next_avl_time

# Calculate next available bike time
for i in df.station_id.unique():
  mask = df['station_id']==i
  data= df[mask]
  all_times= data['datatime_rnd'].values
  avl_times = data[data['new_bikes_available']>0]['datatime_rnd'].values
  df.loc[df['station_id']==i, 'next_avl_bike_time'] = cal_next_avl_time(all_times, avl_times)

# Calculate next available dock time
for i in df.station_id.unique():
  mask = df['station_id']==i
  data= df[mask]
  all_times= data['datatime_rnd'].values
  avl_times = data[data['new_docks_available']>0]['datatime_rnd'].values
  df.loc[df['station_id']==i, 'next_avl_dock_time'] = cal_next_avl_time(all_times, avl_times)

In [ ]:
# Calculate the wait times

# Convert the datatime columns to from string to datetime format
df['datatime_rnd'] = pd.to_datetime(df['datatime_rnd'], format='%Y-%m-%d %H:%M:%S')
df['next_avl_bike_time'] = pd.to_datetime(df['next_avl_bike_time'], format='%Y-%m-%d %H:%M:%S')
df['next_avl_dock_time'] = pd.to_datetime(df['next_avl_dock_time'], format='%Y-%m-%d %H:%M:%S')

# Calculate the bike wait time
df['bike_wait_time'] = (df['next_avl_bike_time'] - df['datatime_rnd']).dt.total_seconds() / 60

# Calculate the dock wait time
df['dock_wait_time'] = (df['next_avl_dock_time'] - df['datatime_rnd']).dt.total_seconds() / 60

df

In [ ]:
df.to_csv('/content/drive/MyDrive/CSCI 5502 Data Mining/01 Project/Data/sample_usable_data_for_initial_analysis.csv',index=False)

# Analyze the change in bike/dock availablity because of replicating nulls
Should we manually adjust the data to maintain continuity? For example, if the 18th data is not available and the 19th is available and you use last week’s for the 18th, then the bike availability should not drastically change by the end of the 18th and the start of the 19th.

In [ ]:
print('Bike and dock availability in 54 stations on Monday (2023-11-06) right after transitioning from replicated data -> actual data:')

new_bikes_of_replaced=[]
new_docks_of_replaced=[]
for i in df_final['station_id'].unique():
  #print(i)
  #print(df_final[(df_final['station_id']==i) & (df_final['date_rnd'] == datetime.date(year=2023, month= 11, day= 6)) & (df_final['hour_rnd']==17) & ((df_final['minute_rnd'] == 36) | (df_final['minute_rnd'] == 39))][['station_id','datatime_rnd','docks_available','bikes_available','new_docks_available','new_bikes_available']])
  new_bikes_of_replaced.append(df_final[(df_final['station_id']==i) & (df_final['date_rnd'] == datetime.date(year=2023, month= 11, day= 6)) & (df_final['hour_rnd']==17) & (df_final['minute_rnd'] == 39)][['new_bikes_available']].values[0][0])
  new_docks_of_replaced.append(df_final[(df_final['station_id']==i) & (df_final['date_rnd'] == datetime.date(year=2023, month= 11, day= 6)) & (df_final['hour_rnd']==17) & (df_final['minute_rnd'] == 39)][['new_docks_available']].values[0][0])
  #print('\n\n')
#df_final[(df_final['date_rnd'] == datetime.date(year=2023, month= 11, day= 6)) & (df_final['datatime_rnd']<'2023-11-06 17:39:00')]
print(new_bikes_of_replaced)
print(new_docks_of_replaced)

Bike and dock availability in 54 stations on Monday (2023-11-06) right after transitioning from replicated data -> actual data:
[0, 2, 0, 1, 5, 0, 1, 1, 0, 2, 4, 0, 0, 0, 6, 0, 2, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 3, 0, 0, 1, 0, 3, 0, 0, 0, 5, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 3, 4, 0, 2, 0, 1, 3]
[0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 4, 0, 7, 0, 0, 0, 1, 0, 2, 0, 5, 2, 1, 4, 6, 0, 0, 4, 0, 0, 0, 0, 1, 0, 0]


The change is drastic in a few stations (can observe a couple of 7, 6, 5 and 4), but for now we can proceed with the analysis.